# Enrich data set

## Libraries and settings

In [158]:
# Libraries
import os
import re
import fnmatch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as py
from geopy.geocoders import Nominatim
from rapidfuzz import process, fuzz

# seaborn graphics settings
sns.set(color_codes=True)

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/wine_analysis


## Importing data

In [159]:
# Read the data to a pandas data frame
df = pd.read_csv('/workspaces/wine_analysis/data/intermediate/wine_data_prepared.csv', sep=',', encoding='utf-8')

# Show first records of data frame
df.head()

,web-scraper-order,web-scraper-start-url,name,price,country_raw,quantity,value,country,name_raw_len,bio,non_alcoholic,premium,wine_category,price_category,quantity_in_cl,price_per_10cl,country_code
0,1734709733-1,https://www.coop.ch/de/weine/alle-weine/c/m_25...,"NATURAPLAN BIO-PROSECCO DOC RAPHAEL DAL BO, EX...",11.95,Italien,75cl,1.59/10cl,Italien,53,1,0,0,SCHAUMWEIN,Mid-Range,75,1.59,IT
1,1734709733-3,https://www.coop.ch/de/weine/alle-weine/c/m_25...,ZÜRICH AOC STAATSSCHREIBER CUVÉE BLANC PRESTIGE,12.70,"Schweiz, 2023",75cl,1.69/10cl,Schweiz,47,0,0,1,WEISSWEIN,Mid-Range,75,1.69,CH
2,1734709733-4,https://www.coop.ch/de/weine/alle-weine/c/m_25...,PROSECCO SUPERIORE DI VALDOBBIADENE CONEGLIANO...,10.95,Italien,75cl,1.46/10cl,Italien,79,0,0,1,SCHAUMWEIN,Mid-Range,75,1.46,IT
3,1734709733-5,https://www.coop.ch/de/weine/alle-weine/c/m_25...,AIGLE LES MURAILLES CHABLAIS AOC H. BADOUX,22.50,"Schweiz, 2023",70cl,3.21/10cl,Schweiz,42,0,0,0,UNCATEGORIZED,Mid-Range,70,3.21,CH
4,1734709733-6,https://www.coop.ch/de/weine/alle-weine/c/m_25...,"CHAMPAGNE AOC CHARLES BERTIN, BRUT",19.50,Frankreich,75cl,2.60/10cl,Frankreich,34,0,0,0,SCHAUMWEIN,Mid-Range,75,2.60,FR


# Enrich Geo Data

In [160]:
# Show all unique countries
df['country'].unique()

#count country values
df['country'].value_counts()

country
Italien        200
Schweiz        140
Frankreich     120
Spanien         40
Portugal        20
Deutschland     20
Name: count, dtype: int64

### Remove rows with Country = Rimuss

In [161]:
# REmove all wines from Rimuss
df = df[df['country'] != 'Rimuss']

df['country'].value_counts()

country
Italien        200
Schweiz        140
Frankreich     120
Spanien         40
Portugal        20
Deutschland     20
Name: count, dtype: int64

### Add country code

In [162]:
# Create a mapping of country names to country codes
country_code_mapping = {
    'Italien': 'IT',
    'Schweiz': 'CH',
    'Frankreich': 'FR',
    'Spanien': 'ES',
    'Portugal': 'PT',
    'Deutschland': 'DE'
}

# Add a new column for country codes using the mapping
df['country_code'] = df['country'].map(country_code_mapping)

# Display the updated dataset with country codes
df[['country', 'country_code']].head()


,country,country_code
0,Italien,IT
1,Schweiz,CH
2,Italien,IT
3,Schweiz,CH
4,Frankreich,FR


### Merge dataset with longitude and latitude

In [163]:
# Load the country coordinates data
coordinates_path = '/workspaces/wine_analysis/data/raw/country_longitude_latitude.csv'
coordinates = pd.read_csv(coordinates_path, sep=',', encoding='utf-8')[['country_code', 'latitude', 'longitude']]

# Merge the datasets on 'country_code'
df = pd.merge(df, coordinates, on='country_code', how='left')
print("Geo data added.")


Geo data added.


## Consumption data

### Filter CSV to only 2019 data, change country code, merge with consumption CSV

In [169]:
# Load the wine consumption data
consumption_path = '/workspaces/wine_analysis/data/raw/wine_consumption_country.csv'
consumption_data = pd.read_csv(consumption_path, sep=',', encoding='utf-8')

# Map two-letter to three-letter country codes
iso_mapping = {
    "CH": "CHE", "CN": "CHN", "CL": "CHL", "ES": "ESP",
    "EE": "EST", "FR": "FRA", "DE": "DEU", "IT": "ITA", "PT": "PRT"
}
df['country_code'] = df['country_code'].map(iso_mapping)

# Filter the consumption data for the year 2019 and merge
consumption_2019 = consumption_data[consumption_data['Year'] == 2019]
df = pd.merge(df, consumption_2019, left_on='country_code', right_on='Code', how='left')

# Drop unnecessary columns and rename for clarity
df = df.drop(columns=['Year', 'Entity', 'Code'], errors='ignore')
df = df.rename(columns={'Consumption': 'consumption_in_l'})
print("Consumption data added.")


Consumption data added.


### Save data to file

In [170]:
# Save the fully enriched dataset
final_output_path = '/workspaces/wine_analysis/data/enriched/wine_data_enriched.csv'
df.to_csv(final_output_path, sep=',', index=False, encoding='utf-8')
print(f"Fully enriched dataset saved at: {final_output_path}")


Fully enriched dataset saved at: /workspaces/wine_analysis/data/enriched/wine_data_enriched.csv


## Alcohol percentage

In [166]:
# Load the new dataset
file_path = '/workspaces/wine_analysis/data/raw/wine_info.csv'
wine_info = pd.read_csv(file_path)

# Keep only the relevant columns
relevant_columns = ['name', 'nation', 'abv']
wine_info = wine_info[relevant_columns]

# Remove non-Latin characters from the 'nation' column
def clean_text(text):
    if isinstance(text, str):
        return re.sub(r'[^\x00-\x7F]+', '', text).strip()
    return text

wine_info['nation'] = wine_info['nation'].apply(clean_text)

# Clean the 'abv' column: Extract the first number if it's a range
def clean_abv(abv_value):
    if isinstance(abv_value, str):
        # Extract the first number before a "~" or any other separator
        match = re.search(r'\d+', abv_value)
        if match:
            return int(match.group(0))  # Convert to integer
    return None  # Return None if no valid number is found

wine_info['abv'] = wine_info['abv'].apply(clean_abv)


# Ensure the 'abv' column is of type integer
wine_info['abv'] = wine_info['abv'].astype('Int64')  # Use 'Int64' for nullable integers

# Drop rows with missing ABV
wine_info = wine_info.dropna(subset=['abv'])

# Filter only the desired countries
allowed_countries = ['Italy', 'Switzerland', 'France', 'Spain', 'Portugal', 'Germany']
wine_info = wine_info[wine_info['nation'].isin(allowed_countries)]

# Manual mapping of countries to their 3-letter codes
country_code_mapping = {
    'Italy': 'ITA',
    'Switzerland': 'CHE',
    'France': 'FRA',
    'Spain': 'ESP',
    'Portugal': 'PRT',
    'Germany': 'DEU'
}

# Add the 3-letter country codes based on the mapping
wine_info['country_code'] = wine_info['nation'].map(country_code_mapping)

# Save the cleaned dataset
output_file_path = '/workspaces/wine_analysis/data/intermediate/cleaned_wine_info.csv'
wine_info.to_csv(output_file_path, index=False, encoding='utf-8')

print(f"Cleaned dataset saved at: {output_file_path}")


Cleaned dataset saved at: /workspaces/wine_analysis/data/intermediate/cleaned_wine_info.csv


### Match Data 

In [167]:
# Load the datasets  
prepared_wine_data = pd.read_csv('/workspaces/wine_analysis/data/enriched/wine_data_enriched.csv')
cleaned_wine_info = pd.read_csv('/workspaces/wine_analysis/data/intermediate/cleaned_wine_info.csv')

# Extract and clean the wine names
prepared_names = prepared_wine_data['name'].str.strip().str.lower()
cleaned_names = cleaned_wine_info['name'].str.strip().str.lower()

# Set a similarity threshold
threshold = 70

# Find matches for all names
matches = []
for name in prepared_names:
    result = process.extractOne(name, cleaned_names, scorer=fuzz.token_set_ratio)
    if result is not None and result[1] >= threshold:
        matched_name = result[0]
        similarity_score = result[1]
        
        # Retrieve `abv` safely
        matched_rows = cleaned_wine_info[cleaned_wine_info['name'].str.strip().str.lower() == matched_name]
        abv = matched_rows['abv'].iloc[0] if not matched_rows.empty else None
        
        matches.append((name, matched_name, similarity_score, abv))

# Convert the results into a DataFrame
matches_df = pd.DataFrame(matches, columns=['Prepared_Wine_Name', 'Matched_Wine_Info_Name', 'Similarity_Score', 'ABV'])

# Drop rows with missing ABV
matches_df = matches_df.dropna(subset=['ABV'])

# Convert ABV to integer
matches_df['ABV'] = matches_df['ABV'].astype(int)

# Save the results to a new file
output_file_path = '/workspaces/wine_analysis/data/intermediate/matched_wines.csv'
matches_df.to_csv(output_file_path, index=False, encoding='utf-8')

print(f"\nNumber of matches found: {len(matches_df)}")
print("Sample matches with ABV:")
print(matches_df.head())



Number of matches found: 300
Sample matches with ABV:
                                  Prepared_Wine_Name  \
0  prosecco superiore di valdobbiadene conegliano...   
1                 champagne aoc charles bertin, brut   
2             prosecco doc vigne dei dogi, extra dry   
3               blu secco dose vino frizzante 6x20cl   
4                   toscana igt rosso villa antinori   

                              Matched_Wine_Info_Name  Similarity_Score  ABV  
0  sommariva, conegliano valdobbiadene prosecco s...         89.523810   11  
1                               angel champagne brut         82.352941   12  
2                        ruffino, prosecco extra dry         80.000000   11  
3                         canei, vino frizzante rose         70.000000    8  
4                                      villa m rosso         91.666667    5  


### Create seperate CSV: wines_with_abv.csv

In [168]:
# Load the necessary files
prepared_wine_data = pd.read_csv('/workspaces/wine_analysis/data/enriched/wine_data_enriched.csv')
matches_df = pd.read_csv('/workspaces/wine_analysis/data/intermediate/matched_wines.csv')

# Check for Null ABV values and drop them
matches_df = matches_df.dropna(subset=['ABV'])

# Standardize the formatting of the names in both DataFrames
prepared_wine_data['name'] = prepared_wine_data['name'].str.strip().str.lower()
matches_df['Prepared_Wine_Name'] = matches_df['Prepared_Wine_Name'].str.strip().str.lower()

# Deduplicate matches_df to ensure unique wine matches
matches_df = matches_df.drop_duplicates(subset=['Prepared_Wine_Name'])

# Filter `prepared_wine_data` to include only wines with matches
filtered_prepared_wine_data = prepared_wine_data[
    prepared_wine_data['name'].isin(matches_df['Prepared_Wine_Name'])
]

# Merge with `matches_df` to include `ABV`
filtered_with_abv = filtered_prepared_wine_data.merge(
    matches_df[['Prepared_Wine_Name', 'ABV']],  # Include only necessary columns
    left_on='name',
    right_on='Prepared_Wine_Name',
    how='inner'  # Use 'inner' to exclude rows without matches
)

# Drop unnecessary columns and ensure ABV is properly formatted
filtered_with_abv = filtered_with_abv.drop(columns=['Prepared_Wine_Name'])
filtered_with_abv['ABV'] = pd.to_numeric(filtered_with_abv['ABV'], errors='coerce').astype(float)  # Convert ABV to float

# Remove duplicates from the final DataFrame
filtered_with_abv = filtered_with_abv.drop_duplicates()

# Save the resulting dataset
output_file_path = '/workspaces/wine_analysis/data/enriched/wines_with_abv.csv'
filtered_with_abv.to_csv(output_file_path, index=False, encoding='utf-8')

print(f"File with valid alcohol content saved at: {output_file_path}")


File with valid alcohol content saved at: /workspaces/wine_analysis/data/enriched/wines_with_abv.csv
